In [1]:
import os
import torch
import matplotlib.pyplot as plt
import tempfile
import shutil
import os
import random
import glob
import nibabel as nib
import numpy as np
from scipy.ndimage import rotate
import csv
import SimpleITK as sitk
#from lungtumormask import mask as tumormask
#from lungmask import mask as lungmask_fun

from monai.utils import first, set_determinism
from monai.transforms import (
    RandFlipd,
    AsDiscreted,
    EnsureChannelFirstd,
    Compose,
    CropForegroundd,
    LoadImaged,
    Orientationd,
    FillHoles,
    RemoveSmallObjects,
    KeepLargestConnectedComponent,
    RandCropByPosNegLabeld,
    ScaleIntensity,
    SaveImaged,
    CenterSpatialCropd,
    SpatialCropd,
    ScaleIntensityRanged,
    Spacingd,
    AsDiscrete,
    SpatialCrop,
    RandSpatialCropd,
    SpatialPadd,
    EnsureTyped,
    EnsureType,
    Invertd,
    DivisiblePadd,
    MapTransform,
    RandWeightedCropd,
    ToTensord,
    Transpose,
    ToTensor,
)
from monai.handlers.utils import from_engine
from monai.networks.nets import UNet,VNet,SwinUNETR,UNETR,DynUNet
from monai.networks.layers import Norm
from monai.metrics import DiceMetric,SurfaceDiceMetric,SurfaceDistanceMetric,HausdorffDistanceMetric
from monai.losses import DiceLoss
from monai.inferers import sliding_window_inference
from monai.data import CacheDataset, DataLoader, Dataset, decollate_batch,pad_list_data_collate
from monai.config import print_config
from monai.apps import download_and_extract


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('device:', device)


if True:
    get_ipython().run_line_magic('matplotlib', 'inline')
    print('plot in line')

device: cuda
plot in line


In [ ]:
def LookSortFiles(root_path, all_patientdir):
    CT_fpaths = []
    lbl_fpaths = []
    lung_fpaths = []

    for patient_path in all_patientdir:
        ct_miss = True
        gtv_miss = True
        lung_miss = True
        for root, dirs, files in os.walk(root_path + patient_path, topdown=False):
            for f in files:
                if True:  # 4D Data local
                    if "ct.nii.gz" in f.lower() and not("ave" in f.lower()):
                        CT_fpaths.append(os.path.join(root_path, patient_path, f))
                        ct_miss = False
                    if "_gtv" in f.lower():
                        lbl_fpaths.append(os.path.join(root_path, patient_path, f))
                        gtv_miss = False
                    if "lungmask.nii.gz" in f.lower() and not("ave" in f.lower()):
                        lung_fpaths.append(os.path.join(root_path, patient_path, f))
                        lung_miss = False
            if gtv_miss:
                for f in files:
                    if "_igtv" in f.lower():
                        lbl_fpaths.append(os.path.join(root_path, patient_path, f))
                        gtv_miss = False
            if ct_miss and lung_miss:
                for f in files:
                    if "ex_ct.nii.gz" in f.lower() and ct_miss:
                        CT_fpaths.append(os.path.join(root_path, patient_path, f))
                        ct_miss = False
                    if 'ex_lungmask.nii.gz' in f.lower() and lung_miss:
                        lung_fpaths.append(os.path.join(root_path, patient_path, f))
                        lung_miss = False
            if ct_miss and lung_miss:
                for f in files:
                    if "mar_ct.nii.gz" in f.lower() and ct_miss:
                        CT_fpaths.append(os.path.join(root_path, patient_path, f))
                        ct_miss = False
                    if 'mar_lungmask.nii.gz' in f.lower() and lung_miss:
                        lung_fpaths.append(os.path.join(root_path, patient_path, f))
                        lung_miss = False
            if ct_miss and lung_miss:
                for f in files:
                    if "ct.nii.gz" in f.lower() and ct_miss:
                        CT_fpaths.append(os.path.join(root_path, patient_path, f))
                        ct_miss = False
                    if 'lungmask.nii.gz' in f.lower() and lung_miss:
                        lung_fpaths.append(os.path.join(root_path, patient_path, f))
                        lung_miss = False
            if gtv_miss and len(files) > 0:
                CT_fpaths.pop()
                ct_miss = True
                lung_fpaths.pop()
                lung_miss = True

    print('ct: ', len(CT_fpaths), 'label: ', len(lbl_fpaths), 'lung: ', len(lung_fpaths))
    if False:
        CreateLungMasks(root_path, CT_fpaths)
        print('Rerun the program')
        exit(1)

    CT_fpaths.sort()
    lbl_fpaths.sort()
    lung_fpaths.sort()

    print(CT_fpaths[-1])
    print(lbl_fpaths[-1])
    print(lung_fpaths[-1])


    if (len(CT_fpaths) != len(lbl_fpaths)) or (len(lbl_fpaths) != len(lung_fpaths)):
        print('Different number of files for each structure')
        exit(1)

    return CT_fpaths, lbl_fpaths, lung_fpaths

In [2]:
##if name
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('device:', device)
num_workers = 0

# root_path = '/data/p308104/Nifti_Imgs_V0/' #UMCG data on peregrine
#root_path = '/data/p308104/MultipleBP/'
#root_path = '/home/umcg/OneDrive/MultipleBreathingP/'
root_path = '/home/umcg/Desktop/AutomaticITV_code/SABR1322_Nifti/'

#preweight_path = '/data/p308104/weights/v9/'
preweight_path = '/home/umcg/Desktop/AutomaticITV_code/weights/v9/'

figures_path = '/home/umcg/Desktop/AutomaticITV_code/figures_folder_i/'

cache = False
lf_select = None # NOT Needed for testing

SelectModel = 0 # 0 Swin  - 1 Dyn
figures_folder_i = figures_path+'figures_SU_l3_v9/'
pretrained_path = preweight_path + 'best_SwinUnet_V9_NBIA_Loss3.pth'
print('LF is ', lf_select, 'Db is UMCG')
name_run = "NameRun"+str(SelectModel) + "LF" + str(lf_select) + "run1"
print(name_run)
main(SelectModel,figures_folder_i,name_run, pretrained_path, cache, num_workers, root_path)


309


In [3]:
##MAIN
all_patientdir = []
all_patientdir = os.listdir(root_path)
all_patientdir.sort()
print(len(all_patientdir),'in',name_run)

CT_fpaths, lbl_fpaths, lung_fpaths = LookSortFiles(root_path, all_patientdir)

#Create data dictionat
data_dicts = [
    {"image": image_name,"lung":lung_name,"label": label_name}
    for image_name,lung_name,label_name in zip(CT_fpaths,lung_fpaths,lbl_fpaths)
]
val_files =  data_dicts[:]
print('train val len:',0,'-',len(val_files))

minmin_CT = -1024 #NBIA
maxmax_CT = 3071 #NBIA

#Create Compose functions for preprocessing of train and validation
set_determinism(seed=0)
image_keys = ["image","lung","label"]
p = .5 #Data aug transform probability
size = 96
image_size = (size,size,size)
pin_memory = True if num_workers > 0 else False  # Do not change


val_transforms = Compose(
    [
        LoadImaged(keys=image_keys),
        EnsureChannelFirstd(keys=image_keys),
        Orientationd(keys=["image","label"], axcodes="RAS"),
        #Spacingd(keys=["image","label"], pixdim=(1,1,1),mode=("bilinear","nearest")),
        ScaleIntensityRanged(keys=["image"], a_min=minmin_CT, a_max=maxmax_CT,b_min=0.0, b_max=1.0, clip=True,),
        Create_sequences(keys=image_keys),
        CropForegroundd(keys=image_keys, source_key="lung",k_divisible = size),
        ToTensord(keys=image_keys),
    ]
)



# Check the images after the preprocessing
if cache:  # Cache
    #train_ds = CacheDataset(data=train_files, transform=train_transforms, cache_rate=1.0, num_workers=num_workers)
    #train_loader = DataLoader(train_ds, batch_size=2, shuffle=True, num_workers=num_workers)
    val_ds = CacheDataset(data=val_files, transform=val_transforms, cache_rate=1.0,
                          num_workers=int(num_workers // 2))
    val_loader = DataLoader(val_ds, batch_size=1, num_workers=int(num_workers // 2), pin_memory=pin_memory)
else:
    #train_ds = Dataset(data=train_files, transform=train_transforms)
    #train_loader = DataLoader(train_ds, batch_size=2, shuffle=True, num_workers=0)
    val_ds = Dataset(data=val_files, transform=val_transforms)
    val_loader = DataLoader(val_ds, batch_size=1, num_workers=0)  # ,collate_fn=pad_list_data_collate)

#Plot Input Volume
#plotInputVolume(val_loader, device)

# Create the model
spatial_dims = 3
max_epochs = 250
in_channels = 1
out_channels = 2  # including background
lr = 1e-3  # 1e-4
weight_decay = 1e-5
T_0 = 40  # Cosine scheduler

task_id = "06"
deep_supr_num = 1  # when is 3 shape of outputs/labels dont match
patch_size = image_size
spacing = [1, 1, 1]
kernels, strides = get_kernels_strides(patch_size, spacing)
if SelectModel:
    print("MODEL Dyn")
    task_id = "06"
    model = DynUNet(
        spatial_dims=3,
        in_channels=in_channels,
        out_channels=out_channels,
        kernel_size=kernels,
        strides=strides,
        upsample_kernel_size=strides[1:],
        norm_name="instance",
        deep_supervision=False,  # when is 3 shape of outputs/labels dont match
        deep_supr_num=deep_supr_num,
    ).to(device)
else:
    print("MODEL SWIN")
    model = SwinUNETR(
        image_size,
        in_channels, out_channels,
        use_checkpoint=True,
        feature_size=48,
        #spatial_dims=spatial_dims
    ).to(device)

#metrics, no definition of :
#NO Loss Function
#NO Optimizer
# Load pretrained model
if pretrained_path is not(None):
    model.load_state_dict(torch.load(pretrained_path, map_location=torch.device(device)))
    print('Using pretrained weights!')
all_metrics = test(figures_folder_i ,image_size,model, val_loader,device,val_files)


#Save csv of metrics
if True:
    f= open(figures_folder_i+str(SelectModel)+'res_UMCGx5_ValidationSET_JusHaus.csv','w', encoding='UTF8')
    writer = csv.writer(f)
    writer.writerow(['Patient','Dice V','Dice P','Sdice V','SDice P','HausD V','HausD P','Diff'])
    writer.writerows(all_metrics)
    f.close()

ct:  307 label:  307 lung:  307
/home/umcg/Desktop/NBIA/NBIA_Nifti_v0/LUNG1-125/LUNG1-125_ct.nii.gz
/home/umcg/Desktop/NBIA/NBIA_Nifti_v0/LUNG1-125/LUNG1-125_ct_GTV-1.nii.gz
/home/umcg/Desktop/NBIA/NBIA_Nifti_v0/LUNG1-125/LUNG1-125_xBP_LungMask.nii.gz


In [4]:
#Create data dictionat
data_dicts = [
    {"image": image_name,"label": label_name,"lung":lung_name}
    for image_name,label_name,lung_name in zip(CT_fpaths,lbl_fpaths,lung_fpaths)
]
val_files = data_dicts[-50:]
print('train val len:',len(val_files))

minmin_CT = -1024 #NBIA
maxmax_CT = 3071 #NBIA

train val len: 50


In [5]:
#class to transpose lung mask
class Create_sequences(MapTransform):
    def __init__(self, keys):
        super().__init__(keys)
        
        print(f"keys to transpose: {self.keys}")

         
    def __call__(self, dictionary):
        dictionary = dict(dictionary)
        for key in self.keys:
            data = dictionary[key]
            if key == 'lung':
                data = np.transpose(data, (0,2,3,1))
                data = rotate(data,270,axes=(1,2),reshape=False)
                data = np.flip(data,1)
                data[data==2] = int(1)
                data[data!=1] = int(0)
            dictionary[key] = data
            
        return dictionary        

In [6]:
#Create Compose functions for preprocessing of train and validation
#set_determinism(seed=0)
image_keys = ["image","label","lung"]
p = .5 #Data aug transform probability
size = 96
image_size = (size,size,size)

val_transforms = Compose(
    [
        LoadImaged(keys=image_keys),
        EnsureChannelFirstd(keys=image_keys),
        Orientationd(keys=["image","label"], axcodes="RAS"),
        #Spacingd(keys=["image","label"], pixdim=(1,1,1),mode=("bilinear","nearest")),
        ScaleIntensityRanged(keys=["image"], a_min=minmin_CT, a_max=maxmax_CT,b_min=0.0, b_max=1.0, clip=True,),
        Create_sequences(keys=image_keys),
        CropForegroundd(keys=image_keys, source_key="lung",k_divisible = size),
        ToTensord(keys=image_keys),
    ]
)

keys to transpose: ('image', 'label', 'lung')


In [7]:
#Check the images after the preprocessing
if False:
    check_ds =Dataset(data=val_files, transform=val_transforms)
    check_loader = DataLoader(check_ds, batch_size=1,num_workers=0)
    if True:
        count = 1
        for batch_data in check_loader:
            #batch_data = first(check_loader)
            image,lung, label = (batch_data["image"][0][0],batch_data["lung"][0][0],batch_data["label"][0][0])
            print(f"px info:{count },image shape: {image.shape},lung shape: {lung.shape}, label shape: {label.shape}")
            count+=1
            for i in range(label.shape[2]):
                if torch.sum(label[:,:,i])>0:
                    plt.subplot(1,3,1),plt.imshow(image[:,:,i]),plt.axis('off')
                    plt.subplot(1,3,2),plt.imshow(label[:,:,i]+lung[:,:,i]),plt.axis('off')
                    plt.subplot(1,3,3),plt.imshow(label[:,:,i]+image[:,:,i]),plt.axis('off')
                    plt.tight_layout(),plt.show()
                    break
            if count>10:
                break


In [8]:
val_ds = Dataset(data=val_files, transform=val_transforms)
val_loader = DataLoader(val_ds, batch_size=1, num_workers=0)#,collate_fn=pad_list_data_collate)


In [9]:
#Create the model
spatial_dims = 3
max_epochs = 100
in_channels = 1
out_channels=2 #including background
model = SwinUNETR(
    image_size, 
    in_channels, out_channels, 
    use_checkpoint=True, 
    feature_size=24,
    #spatial_dims=spatial_dims
).to(device)

#metrics
loss_function = DiceLoss(to_onehot_y=True, sigmoid=True)
optimizer = torch.optim.Adam(model.parameters(), 1e-4)
dice_metric = DiceMetric(include_background=True, reduction="mean")
surfDice_metric_1Class = SurfaceDiceMetric([5],include_background=False)
hausdorff_metric = HausdorffDistanceMetric(include_background=True, reduction="mean")


In [10]:
# Load pretrained model
pretrained_path = '/home/umcg/Desktop/AutomaticITV_code/weights/v8(andOlder)/best_m_MONAI_V3_UMCGWeightsretrainedWithUMCGdata_X2RetrainedwithAITVdata.pth'
#pretrained_path = '/data/p308104/MultipleBP/best_m_MONAI_V3_UMCGWeightsretrainedWithUMCGdata_X2RetrainedwithAITVdata.pth'
if pretrained_path is not(None):
    model.load_state_dict(torch.load(pretrained_path, map_location=torch.device(device)))

    #weight = torch.load(pretrained_path, map_location=torch.device(device))
    #model.load_from(weights=weight)
    print('Using pretrained weights!')

Using pretrained weights!


In [11]:
post_transforms = Compose(
    [
        EnsureType(),
        AsDiscrete(argmax=True,to_onehot=out_channels, threshold=.5),
        FillHoles(applied_labels=1, connectivity=0),
        RemoveSmallObjects(min_size=5, connectivity=3, independent_channels=True),
        #KeepLargestConnectedComponent(applied_labels=None,is_onehot=True,connectivity=3, num_components=3),
   ]
)

post_pred = Compose([EnsureType(), AsDiscrete(argmax=True,threshold=0.5), 
                     ScaleIntensity(minv=0.0, maxv=1.0)])
post_label = Compose([EnsureType(), AsDiscrete(threshold=0.5)],)

In [12]:
#compute metric for current iteration
#Batch x Channel x Height x Width - [B,C,H,W] - 1, 384, 288, 192
#Channel is number of classes
def surfDiceFun_1Class(val_labels,postImg_monai):
    ylabe_BCHW_neg = val_labels[0].cpu().numpy()
    ylabe_BCHW_neg = ylabe_BCHW_neg*-1+1
    ylabe_BCHW_pos = val_labels[0].cpu().numpy()
    ylabe_BCHW = np.concatenate((ylabe_BCHW_neg, ylabe_BCHW_pos),0)
    transpose_monai = Compose([Transpose([3, 0, 1, 2])])
    ylabe_BCHW = transpose_monai(ylabe_BCHW)
    ypred_BCHW = postImg_monai[0].permute(3, 0, 1, 2)
    
    #print(ypred_BCHW.shape,ylabe_BCHW.shape)
    list_surf=[]
    for i in range(ypred_BCHW.shape[0]-4):
        if  np.sum(ylabe_BCHW[i:i+2,1,:,:])>0:# or np.sum(ypred_BCHW[i:i+2,0,:,:])>0:
            surfDice_metric_1Class(y_pred=ypred_BCHW[i:i+2,:,:,:], y=ylabe_BCHW[i:i+2,0:,:,:])
            list_surf.append(surfDice_metric_1Class.aggregate().item())
            #print(list_surf[-1])
            surfDice_metric_1Class.reset()
    return np.mean(list_surf)
#hausdorff_metric(y_pred=postImg_monai, y=val_labels)
#print('hausdorff monai    :',hausdorff_metric.aggregate().item())
#hausdorff_metric.reset()

In [13]:
#Testing the model
count=0
nr_images=8
figsize = (18, 12)
#figures_folder_i = '/data/p308104/MultipleBP_Res/' #Peregrine
figures_folder_i ='/home/umcg/Desktop/AutomaticITV_code/figures_folder_UMCG/' #Local
all_metrics = []
model.eval()
figs = False
with torch.no_grad():
    for val_data in val_loader:
        val_inputs, val_labels = (
            val_data["image"].to(device),
            val_data["label"].to(device),)
        roi_size = image_size
        sw_batch_size = 1
        val_outputs = sliding_window_inference(val_inputs, roi_size, sw_batch_size, model)
        
        postImg_monai = [post_transforms(i) for i in decollate_batch(val_outputs)]
        px = val_data["image"].meta["filename_or_obj"][0].split('/')[-2]
        newGTV_name = val_data["image"].meta["filename_or_obj"][0][:-9]+'aGTV.nii.gz'
        #bp = val_data["image"].meta["filename_or_obj"][0].split('%')[-2][-2:]
        bp = val_data["image"].meta["filename_or_obj"][0].split('_')[-2][:]
        print(px,bp)

        if figs: #AXIAL
            slice_indices = []   
            for i in range(val_inputs.shape[4]):
                if np.sum(val_outputs[0][0,:,:,i])>0:
                    slice_indices.append(i)
            if len(slice_indices) <nr_images:
                slice_indices.append(random.sample(range(1, val_inputs.shape[4]),nr_images-len(slice_indices))[0])
            else:
                slice_indices = random.sample(slice_indices, k=nr_images)
            instance = random.randint(0, val_inputs.shape[0] - 1)
            j = 1
            for i, idx in enumerate(slice_indices):
                j=1+i
                fig = plt.figure('Instance = {}'.format(instance), figsize=figsize)
                plt.subplot(3, nr_images, j),plt.title('AXIAL CT ({})'.format(idx)),plt.imshow(val_inputs.cpu().numpy()[instance, 0, :, :, idx], cmap='gray', vmin=0, vmax=1),plt.axis('off')
                plt.subplot(3, nr_images, nr_images+j),plt.title('Prediction ({})'.format(idx)),plt.imshow(val_inputs.cpu().numpy()[instance, 0, :, :, idx]+val_outputs[0].detach().cpu()[instance, :, :, idx]),plt.axis('off')
                plt.subplot(3, nr_images, 2*nr_images+j),plt.title('postImg Monai ({})'.format(idx)),plt.imshow(val_inputs.cpu().numpy()[instance, 0, :, :, idx]+postImg_monai[0].detach().cpu()[instance, :, :, idx]),plt.axis('off')
            plt.tight_layout(),plt.show()
            if not os.path.exists(os.path.join(figures_folder_i,px)):
                os.makedirs(os.path.join(figures_folder_i,px)) 
            plt.savefig(os.path.join(figures_folder_i,px,bp+'AXIAL final_{}.png'.format(px)))
        if figs: #SAGITAL
            slice_indices = []   
            for i in range(val_inputs.shape[3]):
                if np.sum(val_outputs[0][0,:,i,:])>0:
                    slice_indices.append(i)
            if len(slice_indices) <nr_images:
                slice_indices.append(random.sample(range(1, val_inputs.shape[3]),nr_images-len(slice_indices))[0])
            else:
                slice_indices = random.sample(slice_indices, k=nr_images)
            instance = random.randint(0, val_inputs.shape[0] - 1)
            j = 1
            for i, idx in enumerate(slice_indices):
                j=1+i
                fig = plt.figure('Instance = {}'.format(instance), figsize=figsize)
                plt.subplot(3, nr_images, j),plt.title('SAGITAL CT ({})'.format(idx)),plt.imshow(val_inputs.cpu().numpy()[instance, 0, :, idx,:], cmap='gray', vmin=0, vmax=1),plt.axis('off')
                plt.subplot(3, nr_images, nr_images+j),plt.title('Prediction ({})'.format(idx)),plt.imshow(val_inputs.cpu().numpy()[instance, 0, :, idx,:]+val_outputs[0].detach().cpu()[instance, :,idx,:]),plt.axis('off')
                plt.subplot(3, nr_images, 2*nr_images+j),plt.title('postImg Monai ({})'.format(idx)),plt.imshow(val_inputs.cpu().numpy()[instance, 0, :, idx,:]+postImg_monai[0].detach().cpu()[instance, :,idx,:]),plt.axis('off')
            plt.tight_layout(),plt.show()
            if not os.path.exists(os.path.join(figures_folder_i,px)):
                os.makedirs(os.path.join(figures_folder_i,px)) 
            plt.savefig(os.path.join(figures_folder_i,px,bp+'SAGITAL final_{}.png'.format(px)))
        if figs: #CORONAL
            slice_indices = []   
            for i in range(val_inputs.shape[2]):
                if np.sum(val_outputs[0][0,i,:,:])>0:
                    slice_indices.append(i)
            if len(slice_indices) <nr_images:
                slice_indices.append(random.sample(range(1, val_inputs.shape[2]),nr_images-len(slice_indices))[0])
            else:
                slice_indices = random.sample(slice_indices, k=nr_images)
            instance = random.randint(0, val_inputs.shape[0] - 1)
            j = 1
            for i, idx in enumerate(slice_indices):
                j=1+i
                fig = plt.figure('Instance = {}'.format(instance), figsize=figsize)
                plt.subplot(3, nr_images, j),plt.title('CORONAL CT ({})'.format(idx)),plt.imshow(val_inputs.cpu().numpy()[instance, 0, idx,:,:], cmap='gray', vmin=0, vmax=1),plt.axis('off')
                plt.subplot(3, nr_images, nr_images+j),plt.title('Prediction ({})'.format(idx)),plt.imshow(val_inputs.cpu().numpy()[instance, 0, idx,:,:]+val_outputs[0].detach().cpu()[instance,idx,:,:]),plt.axis('off')
                plt.subplot(3, nr_images, 2*nr_images+j),plt.title('postImg Monai ({})'.format(idx)),plt.imshow(val_inputs.cpu().numpy()[instance, 0,idx,:,:]+postImg_monai[0].detach().cpu()[instance,idx,:,:]),plt.axis('off')
            plt.tight_layout()
            plt.show()
            if not os.path.exists(os.path.join(figures_folder_i,px)):
                os.makedirs(os.path.join(figures_folder_i,px)) 
            plt.savefig(os.path.join(figures_folder_i,px,bp+'CORONAL final_{}.png'.format(px)))

        
        val_outputs = [post_pred(i) for i in decollate_batch(val_outputs)]
        val_labels = [post_label(i) for i in decollate_batch(val_labels)]

        print(
            np.min(val_outputs[0]),
            np.max(val_outputs[0]),
            np.min(val_labels[0]),
            np.max(val_labels[0]),
        )
        
        sum_labels = np.sum(val_labels[0])

        sum_outputs = np.sum(val_outputs[0])

        dif = sum_labels-sum_outputs
        
        
        #compute metric for current iteration
        dice_metric(y_pred=val_outputs, y=val_labels)
        dice0 = dice_metric.aggregate().item()
        print('Scores vanilla:',dice0)
        dice_metric.reset()

        sdice0 = surfDiceFun_1Class(val_labels,val_outputs)
        print('SurfFice vanilla 1 class:',sdice0)
        if True:
            dice_metric(y_pred=postImg_monai[0][1:2,:,:,:], y=val_labels[0])
            dice1 = dice_metric.aggregate().item()
            print('Scores monai    :',dice1)
            dice_metric.reset()
            sdice1 = surfDiceFun_1Class(val_labels,postImg_monai)
            print('SurfFice monai 1 class:',sdice1)
            
        all_metrics.append([val_files[count]['label'].split('/')[-2],dice0,dice1,sdice0,sdice1])
        count+=1

/home/umcg/Desktop/AutomaticITV_code/Aitv_envV0/lib/python3.10/site-packages/torch/utils/checkpoint.py:25: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")


LUNG1-262 v0/LUNG1-262/LUNG1-262
0.0 1.0 0.0 1.0
Scores vanilla: 0.25204935669898987


/home/umcg/Desktop/AutomaticITV_code/Aitv_envV0/lib/python3.10/site-packages/monai/metrics/surface_dice.py:222: UserWarning: the ground truth of class 0 is all 0, this may result in nan/inf distance.
  warnings.warn(f"the ground truth of class {c} is all 0, this may result in nan/inf distance.")
/home/umcg/Desktop/AutomaticITV_code/Aitv_envV0/lib/python3.10/site-packages/monai/metrics/surface_dice.py:224: UserWarning: the prediction of class 0 is all 0, this may result in nan/inf distance.
  warnings.warn(f"the prediction of class {c} is all 0, this may result in nan/inf distance.")


SurfFice vanilla 1 class: 0.16273399482508333
Scores monai    : 0.15301287174224854
SurfFice monai 1 class: 0.16273399482508333
LUNG1-263 v0/LUNG1-263/LUNG1-263
0.0 1.0 0.0 1.0
Scores vanilla: 0.0
SurfFice vanilla 1 class: 0.0
Scores monai    : 0.0
SurfFice monai 1 class: 0.0
LUNG1-264 v0/LUNG1-264/LUNG1-264
0.0 1.0 0.0 1.0
Scores vanilla: 0.7889872789382935
SurfFice vanilla 1 class: 0.7575334875790335
Scores monai    : 0.5932325720787048
SurfFice monai 1 class: 0.7575334875790335
LUNG1-265 v0/LUNG1-265/LUNG1-265
0.0 1.0 0.0 1.0
Scores vanilla: 0.8027952313423157
SurfFice vanilla 1 class: 0.8625119146670213
Scores monai    : 0.6771513223648071
SurfFice monai 1 class: 0.8625119146670213
LUNG1-266 v0/LUNG1-266/LUNG1-266
0.0 1.0 0.0 1.0
Scores vanilla: 0.285881906747818
SurfFice vanilla 1 class: 0.3477842579763954
Scores monai    : 0.17404808104038239
SurfFice monai 1 class: 0.3477842579763954
LUNG1-267 v0/LUNG1-267/LUNG1-267
0.0 1.0 0.0 1.0
Scores vanilla: 0.1091567873954773
SurfFice van

KeyboardInterrupt: 